# A comprehensive guide on DICOM image preprocessing

[DICOM](https://en.wikipedia.org/wiki/DICOM) is an international format to store computer tomography (CT) or magnetic resonance imaging (MRI) data.
Each DICOM file consists of a single scan image and scan meta information which may include the institution name, patient record ID, instance number in the series as well as technical parameters of the scan, e.g. pixel spacing and patient orientation. Full description of DICOM tags can be found in [DICOM Standard Browser](https://dicom.innolitics.com/ciods). Because both the scanned image and its metadata are stored in a single file, there is no chance of inadvertently mixing either of them with data from a different acquisition.

Many challenges on the international data science competition platform [kaggle](www.kaggle.com) are built upon CT or MRI data. Among the most recent is the [RSNA abdominal trauma detection challenge](https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/). In the following, we will use a CT scan from this competition (train_images/14465/21818/135.dcm) to show how to properly load, clean, scale, and save DICOM data. 

## Bits Allocated and Bits Stored

The first thing to note when loading a DICOM image is a possible discrepancy between BitsAllocated (0028,0100) and BitsStored (0028,0101) DICOM tags. 

A typical case is BitsAllocated=16 and BitsStored=12. CT scans often have 4096 levels of gray, which corresponds to 12 bits. The pixel value is then stored in the 12 least significant bits and must be either an unsigned integer or a binary 2's complement integer, when the PixelRepresentation (0028,0103) flag is set. For 2's complement integer pixel values, the sign bit is the HighBit (0028,0102). The remaining 4 bits can be used, for example, to store overlay planes.

Luckily, the organisers of the RSNA abdominal trauma detection challenge [provided](https://www.kaggle.com/code/huiminglin/rsna-2023-abdomen-dicom-standardization) a function that corrects for the BitsAllocated vs BitsStored difference. This function treats the case of PixelRepresentation=1 by basically replacing the leading (BitsAllocated - BitsStored) bits with 1s when pixel values are negative. This ensures that 2's complement pixel values stored in BitsStored - bit representation are properly converted to BitsAllocated - bit representation. 

![DICOM image before and after correcting for bit representation](img/bit_shift.png)

## Hounsfield units



CT pixel values can be transformed to Hounsfield units (HU). HU are specific for CT data and are proportional to the linear attenuation coefficient (radiodensity) in the scanned structures. HU have a clinical value since a quantitative change in radiodensity may be used to characterize the pathology phenotype. 

Note that for a given organ, HU may experience significant patient-to-patient variations. The measured radiodensity also depends on some scan parameters, including KVP (0018,0060) and SliceThickness (0018,0050).

To transform CT pixel values to HU, two DICOM tags are required: RescaleSlope (0028,1053) and RescaleIntercept (0028,1052):

In [ ]:
import pydicom

scan_path = data_dir + 'train_images/9703/39470/485.dcm' # DICOM image path

dcm = pydicom.dcmread(scan_path) # read DICOM image

img = dcm.pixel_array

img = img*dcm.RescaleSlope + dcm.RescaleIntercept # pixel intensities to Hounsfield units

It is always recommended to transform CT pixel values to HU. HU do not only contain information relevant to clinical diagnostics and ultimately to ML tasks such as segmentation and classification, but also help to avoid some normalization problems (e.g. PhotometricInterpretation (0028,0004) issues).

When the mapping between pixels values and HU is not linear, the RescaleIntercept (0028,1052) tag will not be present. Instead, the ModalityLUTSequence (0028,3000) tag must be used to compute the non-linear transformation.

# Windowing

HU range from -1000 HU for air to +~2000 HU for dense bones. For clinical diagnostics and ML applications, only a certain "window" within this range is typically of interest.

Windowing is basically performed by clipping all values beyond the window range. For instance, a typical abdominal CT window is centered at 50 HU and is 400 HU wide. To apply this window, all HU below -150 are set to -150 and all HU above +250 are set to +250.

The values within the window range can then be linearly mapped to a (0,1) scale for a better visualization or to satisfy requirements for the neural network input. Note that this way of mapping differs from MinMax scaling which converts pixel intensity values to a (0,1) range for each single image. In MinMax scaling, pixels values are normalized against the brightness range specific to each scan. As a result, HU are transformed differently for each image, which might degrade classification performance.

Here is how windowing can be coded:

In [ ]:
import numpy as np

WindowCenter, WindowWidth = 50, 400 # typical HU window for abdominal scan

vmin = WindowCenter - WindowWidth/2 # -150
vmax = WindowCenter + WindowWidth/2 # 250
        
img = np.clip(img, vmin, vmax) # clip
        
img = (img - vmin)/(vmax - vmin) # stretch

![CT scann with different windows applied](img/windowing.png)

# Cleaning

CT scans often contain elements irrelevant to diagnostics, s.t. a patient table or an emergency button with cable. These elements impact embeddings and may make predictions less reliable.

To remove the annoying stuff, we will first define a mask by choosing all pixels above the bottom edge of the abdominal window (HU=-150). Afterwards, we use the morphology.remove_small_objects function from the scikit-image package to remove all objects smaller than 12000 mm2, which we emprically found to be the minimal body cross-sectional area on the scan.

Windowing or clipping pixel intensities at HU=-150 makes lungs appear as holes on the image mask since they are normally filled with air (HU = -1000). We will then apply the skimage.morphology.remove_small_holes function to fill in such areas as well as smaller holes that may appear on the mask.

In [ ]:
import skimage

crop_HU_thr = -150 # bottom edge of the abdominal window
min_body_area_mm2 = 12000 # minimal body cross-sectionl area (e.g. legs), defined heuristically
max_hole_area_mm2 = 10000 # maximal area of the hole in body mask to fill (e.g. lungs), defined heuristically
background_HU = -1000 # HU of air

pixel_size_mm2 = np.prod(dcm.PixelSpacing) # actual pixel size on the image

keep_mask = img>crop_HU_thr

# remove small objects (patient table, arm, etc) 
mask1 = skimage.morphology.remove_small_objects(keep_mask, min_size=int(min_body_area_mm2/pixel_size_mm2))
        
# remove holes in the body cross-section (lungs, etc)
mask2 = skimage.morphology.remove_small_holes(mask1, area_threshold=int(max_hole_area_mm2/pixel_size_mm2))
    
img[~keep_mask] = background_HU # erase everything outside the body mask by assigning to background

![Removing irrelevant objects with masking](img/masks.png)

Note that this approach implies that windowing is applied after cleaning:

![Applying windowing after cleaning](img/clean_windowed.png)

# Cropping and Resizing

The image often has a lot of black space after cleaning. It often makes sense to crop out the relevant part:

In [ ]:
import cv2

x, y, w, h = cv2.boundingRect((mask2*255).astype('uint8')) # obtain rectangular area around the mask

img = img[y:y+h, x:x+w]

When the resulting image must have fixed dimensions (for example, for a CNN input), a simple resizing operation can be applied:

In [ ]:
output_size = (256, 256) # required output dimensions, px

img = cv2.resize(img, dsize=output_size)

The size of a single pixel (pixel spacing) can vary from one patient to another. When the data amount is small, it can be beneficial to keep the pixel spacing fixed on the output image, say 1.5mm x 1.5mm. In this case, the resizing operation gets a bit more complicated: we must first scale the pixel dimensions, then place the initial image onto the background of desired size.

In [ ]:
pixel_spacing = (1.5, 1.5) # required output spacing, mm
height_new, width_new = (256, 256) # required output size, px

img = cv2.resize(img, dsize=None, fx = dcm.PixelSpacing[1]/pixel_spacing[1], fy=dcm.PixelSpacing[0]/pixel_spacing[0]) # scale pixels
        
height, width = img.shape # actual dimensions

img = img[max((height-height_new)//2,0):(height+height_new)//2,
        max((width-width_new)//2,0):(width+width_new)//2] # use the central part of the image if actual dimensions are larger than required dimensions
        
height, width = img.shape # recompute dimensions after cropping the central part

# position of the old image inside the new image
x0 = width_new//2 - width//2 
y0 = height_new//2 - height//2
                
img_new = np.ones((height_new, width_new))*img.min() # position of the old image inside the new image
        
img_new[y0:y0 + height, x0:x0 + width] = img # place the old image in the center of the new image

![Fixed dimensions vs Fixed spacing approach](img/resizing.png)

# Conversion to PNG

DICOM images are often furnished without any compression. In addition, they usually contain HU units on the full scale, before windowing. As a result, storing a DICOM image needs a lot of space while loading a DICOM image requires a lot of time. To train neural networks faster and save storage space, DICOM images can be converted to PNG files.

To ensure optimal compression, we choose the minimal png depth that is sufficient to encode N=WindowWidth distinct HU values. We then map the pixel values from [0,1] back to the [0,WindowWidth] scale and save as a PNG image:

In [ ]:
import png

WindowCenter, WindowWidth = 50,400

bitdepth = np.ceil(np.log2(WindowWidth)).astype(int)

with open('img.png', 'wb') as f:
    png_writer = png.Writer(img.shape[1], img.shape[0], bitdepth=bitdepth)  
    png_writer.write(f, (img*WindowWidth).astype(int))

This operation reduced the initial image file from 242Kb to just 41Kb, which is around 6x.

For convinience, I created a [dicomprocessing module](dicomprocessing/) that performs all the described operations.